In [1]:
%timeit

import os
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress Tensorflow verbose prints
warnings.simplefilter(action='ignore', category=FutureWarning)

%load_ext autoreload
%autoreload 2

from utils import *
from models import *
import matplotlib.pyplot as plt
from scipy import spatial
from pathlib import Path
import tensorflow as tf
from tqdm import tqdm
import pandas as pd
import numpy as np
import itertools
import random
import math
import cv2

models_path = Path.cwd() / 'models' / 'sphereface_20_keras'
data_path = Path.cwd() / 'datasets'
train_dataset_path = data_path / 'CASIA-WebFace-112x96'
test_dataset_path = data_path / 'lfw_112x96'

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

## Load data

In [38]:
%%time
print("load test data")
test_dataframe = pd.read_csv(str(data_path / 'test.csv')) 
print("load train data")
train_dataframe = pd.read_csv(str(data_path / 'train.csv')) 

load test data
load train data
Wall time: 55.4 s


In [42]:
train_dataframe.head()
train_dataframe.columns

Index(['Unnamed: 0', 'fileL', 'fileR', 'flag'], dtype='object')

In [40]:
test_dataframe.head()

,Unnamed: 0,fileL,fileR,flag
0,0,Abel_Pacheco\Abel_Pacheco_0001.jpg,Abel_Pacheco\Abel_Pacheco_0004.jpg,1
1,1,Akhmed_Zakayev\Akhmed_Zakayev_0001.jpg,Akhmed_Zakayev\Akhmed_Zakayev_0003.jpg,1
2,2,Akhmed_Zakayev\Akhmed_Zakayev_0002.jpg,Akhmed_Zakayev\Akhmed_Zakayev_0003.jpg,1
3,3,Amber_Tamblyn\Amber_Tamblyn_0001.jpg,Amber_Tamblyn\Amber_Tamblyn_0002.jpg,1
4,4,Angela_Bassett\Angela_Bassett_0001.jpg,Angela_Bassett\Angela_Bassett_0005.jpg,1


## Train generator

In [5]:
# https://machinelearningmastery.com/how-to-load-large-datasets-from-directories-for-deep-learning-with-keras/
# https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
# https://stackoverflow.com/questions/49404993/keras-how-to-use-fit-generator-with-multiple-inputs

def preprocess_image(image):
    image = (image - 127.5) / 128
    return image


def train_generator(train_dataframe, batch_size_):
    class_mode_ = "binary"
    generator = ImageDataGenerator(preprocessing_function=preprocess_image, validation_split=0.0)
    
    train_generator_X1 = generator.flow_from_dataframe(
                             dataframe=train_dataframe,
                             directory=str(train_dataset_path) + "/",
                             x_col="fileL",
                             y_col="flag",
                             subset="training",
                             batch_size=batch_size_,
                             seed=42,
                             shuffle=True,
                             class_mode=class_mode_,
                             color_mode='rgb',
                             target_size=(112, 96))
    
    train_generator_X2 = generator.flow_from_dataframe(
                             dataframe=train_dataframe,
                             directory=str(train_dataset_path) + "/",
                             x_col="fileR",
                             y_col="flag",
                             subset="training",
                             batch_size=batch_size_,
                             seed=42,
                             shuffle=True,
                             class_mode=class_mode_,
                             color_mode='rgb',
                             target_size=(112, 96))
    while True:
        X1i = train_generator_X1.next()
        X2i = train_generator_X2.next()
        yield [X1i[0], X2i[0]], X1i[1]

## Test generator

In [6]:
def test_generator(test_dataframe, batch_size):
    class_mode_ = "binary"
    test_datagen = ImageDataGenerator(
                             preprocessing_function=preprocess_image) 
    test_generator_X1 = test_datagen.flow_from_dataframe(
                             dataframe=test_dataframe,
                             directory=str(test_dataset_path) + "/",
                             x_col="fileL",
                             y_col="flag",
                             batch_size=batch_size,
                             seed=42,
                             shuffle=False,
                             class_mode=class_mode_,
                             color_mode='rgb',
                             target_size=(112,96))

    test_generator_X2 = test_datagen.flow_from_dataframe(
                             dataframe=test_dataframe,
                             directory=str(test_dataset_path) + "/",
                             x_col="fileR",
                             y_col="flag",
                             batch_size=batch_size,
                             seed=42,
                             shuffle=False,
                             class_mode=class_mode_,
                             color_mode='rgb',
                             target_size=(112,96))
    while True:
        X1i = test_generator_X1.next()
        X2i = test_generator_X2.next()
        yield [X1i[0], X2i[0]], X1i[1]

## Model

In [11]:
from tensorflow.keras.layers import Conv2D, Add, Activation, PReLU, Dense, Input, ZeroPadding2D, Lambda, GlobalAveragePooling2D, Dot 
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CosineSimilarity
from tensorflow.keras.optimizers import RMSprop
# from keras.engine.topology import Layer
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow import keras

def conv_3_block(input, filters):
    x = ZeroPadding2D(padding=(1, 1))(input)
    x = Conv2D(filters, 3, strides=2, padding='valid', kernel_initializer='glorot_uniform')(x)
    r1 = PReLU()(x)

    x = ZeroPadding2D(padding=(1, 1))(r1)
    x = Conv2D(filters, 3, strides=1, padding='valid', kernel_initializer=TruncatedNormal(stddev=0.01))(x)
    r2 = PReLU()(x)

    x = ZeroPadding2D(padding=(1, 1))(r2)
    x = Conv2D(filters, 3, strides=1, padding='valid', kernel_initializer=TruncatedNormal(stddev=0.01))(x)
    r3 = PReLU()(x)

    x = Add()([r1, r3])
    return x 

def conv_2_block(input, filters):
    x = ZeroPadding2D(padding=(1, 1))(input)
    x = Conv2D(filters, 3, strides=1, padding='valid', kernel_initializer=TruncatedNormal(stddev=0.01))(x)
    x = PReLU()(x)

    x = ZeroPadding2D(padding=(1, 1))(x)
    x = Conv2D(filters, 3, strides=1, padding='valid', kernel_initializer=TruncatedNormal(stddev=0.01))(x)
    x = PReLU()(x)

    x = Add()([input, x])
    return x

def sphereface20(input_shape):
    input = Input(shape=input_shape)
    x = conv_3_block(input, 64)
    x = conv_3_block(x, 128)
    x = conv_2_block(x, 128)
    x = conv_3_block(x, 256)
    x = conv_2_block(x, 256)
    x = conv_2_block(x, 256)
    x = conv_2_block(x, 256)
    x = conv_3_block(x, 512)
    
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, kernel_initializer='glorot_uniform')(x)
    
    model = Model(input, x)
    return model

# https://keras.io/examples/mnist_siamese/
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

# https://stackoverflow.com/questions/51003027/computing-cosine-similarity-between-two-tensors-in-keras
def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)


# network definition
input_shape = (112, 96, 3)
# base_network = sphereface20(input_shape)
base_network = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape,
                                   pooling='avg')

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

# serialize model to JSON
folder_path = Path.cwd() / 'models' / 'sphereface_20_keras'
folder_path.mkdir(parents=True, exist_ok=True)
# model_path = str(folder_path / 'sphereface_20.json')
# model_json = base_network.to_json()
# with open(model_path, "w") as json_file:
#     json_file.write(model_json)

## Model FaceNet

In [12]:
def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def custom_loss(yTrue,yPred):
    return K.sum(K.log(yTrue) - K.log(yPred))

# # def cosine_similarity(y_true, y_pred):
# #     y = tf.constant([c1,c2])
# #     x = K.l2_normalize(y_true, -1)
# #     y = K.l2_normalize(y_pred, -1)
# #     s = K.mean(x * y, axis=-1, keepdims=False) * 10
# #     return s

# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     0.01,
#     decay_steps=10000,
#     decay_rate=0.96,
#     staircase=True)
# # optimizer_ = keras.optimizers.SGD(learning_rate=lr_schedule)

# # rms_ = RMSprop()
# optimizer_ = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-07)
# # loss_ = custom_loss
# # loss_ = CosineSimilarity(axis=-1, name='cosine_similarity')

# model.compile(loss=contrastive_loss, optimizer=optimizer_, metrics=[accuracy])
# # model.compile(loss=loss_, optimizer=rms_, metrics=['accuracy'])

In [13]:
# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])

## Train

In [14]:
sd=[]
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = [1,1]

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        sd.append(step_decay(len(self.losses)))
        print(', lr:', step_decay(len(self.losses)))

learning_rate = 0.01
decay_rate_ = 5e-6
momentum_ = 0.9
batch_size_ = 32
images_per_epoch_ = 100000

sgd = keras.optimizers.SGD(lr = learning_rate,
                           momentum = momentum_, 
                           decay = decay_rate_, 
                           nesterov=False)
model.compile(loss = contrastive_loss, 
              optimizer = sgd, 
              metrics = [accuracy])

In [15]:
train_g = train_generator(train_dataframe, batch_size_)
test_g = test_generator(test_dataframe, batch_size_)

In [ ]:
def scheduler(epoch, lr):
    momentum=momentum_      # 0.8
    decay_rate=decay_rate_  # 2e-6
    lr = learning_rate      # 0.0005
    return lr
    
def step_decay(losses):
    lrate=learning_rate
    momentum=momentum_
    decay_rate=decay_rate_
    return lrate

history = LossHistory()
lrate = keras.callbacks.LearningRateScheduler(scheduler)
callbacks_ = [history, lrate]
# callbacks_ = [history]
initial_epoch_ = 0
epochs_ = 10

try:
    hist = model.fit(train_g,
                     steps_per_epoch=images_per_epoch_ // batch_size_,
                     epochs=epochs_,
                     validation_data=test_g,
                     validation_steps=len(test_dataframe) // batch_size_,
                     callbacks=callbacks_,
                     initial_epoch=initial_epoch_,
                     shuffle=True)
except KeyboardInterrupt:
    save_path = str(models_path / 'sphereface_20_interrupted.h5')
    model.save_weights(save_path)


Epoch 1/10
Found 46852992 validated image filenames belonging to 2 classes.
Found 46852992 validated image filenames belonging to 2 classes.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2552/3125 [=======================>......] - ETA: 8:06 - loss: 1.7943 - accuracy: 0.5228

In [ ]:
save_path = str(models_path / 'sphereface_20_new.h5')
model.save_weights(save_path)

In [ ]:
weights_path = str(folder_path / 'sphereface_20_8372.h5')
model.load_weights(weights_path)

## Predict

In [ ]:
pred = model.predict_generator(test_generator(test_dataframe, batch_size_), 
                               steps = len(test_dataframe) // batch_size_,
                               verbose=1)